In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from face_segmenter import segment_image,rgb2gray, next_pyramid_step
import matplotlib.pyplot as plt

# Entrenamiento

In [2]:
csv_path = "/media/nacho/datasets/CelebA/list_bbox_celeba.csv"
images_dir = "/media/nacho/datasets/CelebA/img_align_celeba/img_align_celeba"

images_properties = pd.read_csv(csv_path,sep=",",nrows=100000)

In [3]:
x1 = 45
y1 = 85
w1 = 90
h1 = 90
n_clusters = 64

In [4]:
def sharpen_image(image):
    sharpening = np.array([[-1,-1,-1],[-1,9.5,-1],[-1,-1,-1]])
    image = cv2.filter2D(image,-1,sharpening)
    return image

In [5]:
def change_brightness(image):
    bright = np.ones(image.shape,dtype=int) * (np.random.random() * 0.6 - 0.3)
    return np.clip(image + bright,0,1) 

In [6]:
faces_list = []
faces_list_augmented = []
for _,image in images_properties.iterrows():
    image_path,bbox1,bbox2,bbox3,bbox4 = image.tolist()
    image_path = os.path.join(images_dir,image_path)

    img = cv2.imread(image_path)
    img = rgb2gray(img) / 255
    face = img[y1:y1+h1,x1:x1+w1]
    
    face = cv2.resize(face,(32,32))
    face_f = cv2.flip(face,1)
    
    faces_list.append(face)    
    faces_list.append(face_f)

    faces_list_augmented.append(sharpen_image(face))
    faces_list_augmented.append(sharpen_image(face_f))

    faces_list_augmented.append(change_brightness(face))
    faces_list_augmented.append(change_brightness(face_f))


In [7]:
plt.imshow(faces_list_augmented[79],cmap="gray")

In [8]:
tile_list = []
for face in faces_list:
    tiles,_ = segment_image(face,8)
    tile_list.append(tiles)

tiles_np = np.concatenate(tile_list)

In [9]:
tile_list = []
for face in faces_list_augmented:
    tiles,_ = segment_image(face,8)
    tile_list.append(tiles)

tiles_augmented_np =  np.concatenate(tile_list)

In [10]:
natural_images_path = "/media/nacho/datasets/natural_images"

natural_images_list = []

for data_folder in os.listdir(natural_images_path):
    if data_folder == "person": continue
    for image in os.listdir(os.path.join(natural_images_path,data_folder)):
        img_npy = cv2.imread(os.path.join(natural_images_path,data_folder,image))
        img_npy = rgb2gray(img_npy)
        """for i in range(10):
            img_npy = next_pyramid_step(img_npy)"""
        subregiones,_ = segment_image(img_npy,8)
        natural_images_list.append(subregiones)

natural_images = np.concatenate(natural_images_list)

In [11]:
train_tiles_np = np.concatenate([tiles_np,natural_images])

NameError: name 'natural_images' is not defined

In [10]:
#pipe = Pipeline([('kmeans', KMeans(n_clusters,n_init=1,verbose=True))])
pipe = KMeans(n_clusters=n_clusters,n_init=5)
pipe.fit(train_tiles_np.reshape(-1,64))

NameError: name 'train_tiles_np' is not defined

In [12]:
from joblib import dump
import skops.io as sio
with open("models/pipe_kmeans64.sio","+wb") as f:
    obj = sio.dumps(pipe)
    f.write(obj)

In [13]:
from joblib import load
with open("models/pipe_pca099_kmeans64.sio","br") as f:
    obj = f.read()
    unknown_types = sio.get_untrusted_types(data=obj)
    pipe = sio.loads(obj,trusted=unknown_types)


## Conteo

In [30]:
y_prime = pipe.predict(np.concatenate([tiles_np,tiles_augmented_np]).reshape(-1,64))
y_prime_pos = y_prime.reshape(-1,16)
P_q_cara,_ = np.histogram(y_prime,bins=range(0,n_clusters+1))

In [31]:
P_pos_q_cara_list = []
for i in range(y_prime_pos.shape[1]):
    P_pos_q_cara_i, _ = np.histogram(y_prime_pos[:,i],bins=range(0,n_clusters+1)) 
    P_pos_q_cara_list.append(P_pos_q_cara_i)

P_pos_q_cara = np.stack(P_pos_q_cara_list)

In [32]:
np.save("counts/pipe_kmeans64/P_q_cara.npy",P_q_cara)
np.save("counts/pipe_kmeans64/P_pos_q_cara.npy",P_pos_q_cara)

In [33]:
natural_images_path = "/media/nacho/datasets/natural_images"

natural_images_list = []

for data_folder in os.listdir(natural_images_path):
    if data_folder == "person": continue
    for image in os.listdir(os.path.join(natural_images_path,data_folder)):
        img_npy = cv2.imread(os.path.join(natural_images_path,data_folder,image))
        img_npy = rgb2gray(img_npy)
        for i in range(10):
            img_npy = next_pyramid_step(img_npy)
            subregiones,_ = segment_image(img_npy,8)
            natural_images_list.append(subregiones)

natural_subregions = np.concatenate(natural_images_list,axis=0)
y_naturales_prime = pipe.predict(natural_subregions.reshape(-1,64))
P_q_nocara,_ = np.histogram(y_naturales_prime,bins=range(0,n_clusters+1))
np.save("counts/pipe_kmeans64/P_q_nocara.npy",P_q_nocara)

## Calcular el resultado

In [33]:
P_norm_q_nocara = P_q_nocara / P_q_nocara.sum()

In [34]:
P_norm_q_cara = P_q_cara / P_q_cara.sum()

In [35]:
P_pos_q_cara.sum(axis=0)

array([32094, 54222, 25041, 45680, 31461, 32514, 30685, 18467, 19298,
       16893, 24972, 37314, 14561, 16127, 13171, 46596, 17023, 15332,
       15051, 21556, 13576,  6964, 15547, 35692, 14952, 19123, 54018,
       11622, 15052,  7972, 16764, 37307, 34299, 28803, 52571, 12367,
       18361, 40813, 24377, 15683, 35383, 14224, 25451, 17477, 33925,
       47754, 14303, 14099, 16891, 30699, 10569, 30346, 18090, 48605,
       25200, 12136, 11798, 13123, 40996, 49845, 27492, 29572, 14042,
       14059])

In [36]:
P_norm_pos_q_cara = P_pos_q_cara / P_pos_q_cara.sum(axis=0)

In [37]:
P_norm_pos_q_cara.shape

(16, 64)

In [38]:
test = np.asarray([10,20,30,40,50,60])

In [39]:
test

array([10, 20, 30, 40, 50, 60])

In [41]:
P_norm_q_nocara[20],P_norm_q_nocara[30],P_norm_q_nocara[40]

(0.008417508417508417, 0.0016835016835016834, 0.005050505050505051)

In [42]:
den = np.prod(P_norm_q_nocara[test]/ (16*64)) 

In [43]:
np.prod(P_norm_pos_q_cara[np.arange(0,64,1,dtype=int),test])

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (64,) (6,) 

In [44]:
num = np.prod(P_norm_pos_q_cara[np.arange(0,6,1,dtype=int),test]) * np.prod(P_norm_q_cara[test])